In [ ]:
import pandas as pd
import datetime
import csv
import re
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from tensorflow.keras import models, layers, preprocessing as kprocessing
import sys, traceback
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from datetime import datetime
import matplotlib.pylab as plt

%pylab inline

In [ ]:
dataset = pd.read_csv("data/taeix_lstm_input.csv")
values = dataset.values
dataset.head(4)

In [ ]:
values = values.astype('float32')

In [ ]:
# normalizing input features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
scaled =pd.DataFrame(scaled)

In [ ]:
values.shape

In [ ]:
size = len(scaled)
train_size = int(size * 0.8)
test_size = size - train_size
c_size = len(scaled[0])
X_train = scaled[:train_size]
X_test = scaled[train_size:]
y_train = X_train.iloc[:,-1:]
y_test = X_test.iloc[:,-1:]
X_train = X_train.iloc[: , :-1]
X_test = X_test.iloc[: , :-1]
type(X_train)

In [ ]:
X_train2 = X_train.values
X_train2 = X_train2.reshape((X_train2.shape[0], 1, X_train2.shape[1]))
#y_train2 = y_train.values
#y_train2 = y_train2.reshape((y_train2.shape[0], 1, y_train2.shape[1]))
X_test2 = X_test.values
X_test2 = X_test2.reshape((X_test2.shape[0], 1, X_test2.shape[1]))
#y_test2 = y_test.values
#y_test2 = y_test2.reshape((y_test2.shape[0], 1, y_test2.shape[1]))

In [ ]:
X_test

In [ ]:
print(X_train2.shape)
print(y_train2.shape)
print(y_test.shape)
print(y_test2.shape)

In [ ]:
u = X_train2.shape[2]
i = X_train2.shape[1]
print(u,i)
lstm_model = models.Sequential()

lstm_model.add(layers.LSTM(units=50, return_sequences = True, activation='tanh', input_shape=(i, u)))
lstm_model.add(layers.Dropout(0.2))
#print("2")
lstm_model.add(layers.LSTM(units=50, return_sequences = True))
lstm_model.add(layers.Dropout(0.2))

lstm_model.add(layers.Dense(1))
lstm_model.compile(optimizer='adam', loss='mse')
#print("6")
lstm_model.summary()

In [ ]:
lstm_model.fit(X_train2, y_train, epochs = 10, batch_size = 32)

In [ ]:
predicted_value= lstm_model.predict(X_test2)
predicted_value.shape

In [ ]:
yhat = list()
for p in predicted_value:
    yhat.append(p[0])
#yhat

In [ ]:
X_test2 = X_test2.reshape((X_test2.shape[0], X_test2.shape[2]))

#print(X_test2.shape)

In [ ]:
inv_yhat = concatenate(( X_test2[:, :], predicted_value[:,0]), axis=1)
#inv_yhat.shape

In [ ]:
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,-1]
#inv_yhat

In [ ]:
y_test2 = y_test.values
# invert scaling for actual
y_test2 = y_test2.reshape((len(y_test2), 1))
inv_y = concatenate((X_test2[:, :], y_test2), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,-1]
#inv_y


In [ ]:
# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)


In [ ]:
actual = y_test.reset_index(drop=True)
actual

In [ ]:
pred = pd.DataFrame(predicted_value[:,0,0], columns=['21'])
pred

In [ ]:
plt.plot(pred, color= 'red')
plt.plot(actual, color='green')
plt.title("Opening price of stocks sold")
plt.xlabel("Time (latest-> oldest)")
plt.ylabel("Stock Opening Price")
plt.show()

In [ ]:
x = scaler.inverse_transform(y_test)
x